In [4]:
import Pkg


In [5]:
Pkg.add(["CSV", "DataFrames", "StatsModels", "GLM", "Random", "Downloads", "CategoricalArrays", "Statistics", "PrettyTables"])
Pkg.add("MLJ")
Pkg.add("MLJScikitLearnInterface") # Para OLS, Lasso, RF
Pkg.add("MLJFlux") # Para Redes Neuronales
Pkg.add("Flux")
Pkg.add("Crayons")

   Resolving package versions...
     Project No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Manifest.toml`
   Resolving package versions...
     Project No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\user2\.julia\environments\v1.12\Manifest.toml`
   Resolving pac

In [6]:
using CSV, DataFrames, StatsModels, GLM, Random, Downloads, CategoricalArrays, Statistics, PrettyTables
using MLJ, MLJScikitLearnInterface, MLJFlux, Flux

In [7]:
output_dir = "C:/Users/user2/Documents/GitHub/DML_NN/julia/output"
mkpath(output_dir)
output_path = joinpath(output_dir, "comparison_table.csv")

"C:/Users/user2/Documents/GitHub/DML_NN/julia/output\\comparison_table.csv"

# carga de datos

In [8]:
## Limpieza de datos
nombres = [
    "abdt", "tg", "inuidur1", "inuidur2", "female", "black", "hispanic", 
    "othrace", "dep", "q1", "q2", "q3", "q4", "q5", "q6", "recall", 
    "agelt35", "agegt54", "durable", "nondurable", "lusd", "husd", "muld"
]

23-element Vector{String}:
 "abdt"
 "tg"
 "inuidur1"
 "inuidur2"
 "female"
 "black"
 "hispanic"
 "othrace"
 "dep"
 "q1"
 ⋮
 "q6"
 "recall"
 "agelt35"
 "agegt54"
 "durable"
 "nondurable"
 "lusd"
 "husd"
 "muld"

In [9]:
url = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/penn_jae.dat"
file_path = "penn_jae.dat"  ## C:\Users\user2\Documents\GitHub\DML_NN\input\penn_jae.dat
if !isfile(file_path)
    Downloads.download(url, file_path)
end

In [10]:
df = CSV.read(file_path, DataFrame, skipto=2, header=nombres, delim=' ', ignorerepeated=true)

Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,10635,2,7,3,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
3,10551,5,18,6,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
4,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6,10544,6,7,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
7,10845,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
8,10670,3,3,3,1,0,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,1
9,10768,3,28,11,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


## limpiar

In [11]:
df_cleaned = filter(row -> row.tg == 0 || row.tg == 4, df)

Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,10607,4,9,9,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
5,10831,0,27,27,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0
6,10845,0,27,27,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
7,10831,0,9,9,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0
8,10859,0,27,27,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
9,10516,0,15,15,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [12]:
df_cleaned[!, :T4] = (df_cleaned.tg .== 4)
df_cleaned[!, :y] = log.(Float64.(df_cleaned.inuidur1))
df_cleaned.y = replace(df_cleaned.y, -Inf => missing)

dropmissing!(df_cleaned, [:y])

Row,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,q2,q3,q4,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4,y
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Bool,Float64
1,10824,0,18,18,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,false,2.89037
2,10824,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,false,0.0
3,10747,0,27,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,false,3.29584
4,10607,4,9,9,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,true,2.19722
5,10831,0,27,27,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,false,3.29584
6,10845,0,27,27,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,false,3.29584
7,10831,0,9,9,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,false,2.19722
8,10859,0,27,27,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,false,3.29584
9,10516,0,15,15,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,false,2.70805


In [13]:
df_cleaned[!, :dep_1] = (df_cleaned.dep .== 1)
df_cleaned[!, :dep_2] = (df_cleaned.dep .== 2)

5099-element BitVector:
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 1
 0
 1
 1
 0
 0

In [14]:
# Definir lista de features 'x'
feature_list = [
    :female, :black, :othrace,
    :dep_1, :dep_2,  # :dep_0 es referencia
    :q2, :q3, :q4, :q5, :q6, # :q1 es referencia
    :recall, :agelt35, :agegt54,
    :durable, :nondurable, :lusd, :husd
]

17-element Vector{Symbol}:
 :female
 :black
 :othrace
 :dep_1
 :dep_2
 :q2
 :q3
 :q4
 :q5
 :q6
 :recall
 :agelt35
 :agegt54
 :durable
 :nondurable
 :lusd
 :husd

In [15]:
# Definir x, y, d
x = df_cleaned[!, feature_list]
y = df_cleaned.y
d = Float64.(df_cleaned.T4) # Convertir Boolean a Float64

x = coerce(x, Count => Continuous)
println(first(x, 5))

5×17 DataFrame
 Row │ female   black    othrace  dep_1    dep_2    q2       q3       q4       q5       q6       recall   agelt35  agegt54  durable  nondurable  lusd     husd    
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64     Float64  Float64 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     0.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      0.0         0.0      0.0      1.0
   2 │     0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      0.0         0.0      1.0      0.0
   3 │     0.0      0.0      0.0      0.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0         0.0      1.0      0.0
   4 │    

## Definición de Funciones DML

In [16]:
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
NeuralNetworkRegressor = @load NeuralNetworkRegressor pkg=MLJFlux verbosity=0
LogisticCVClassifier = @load LogisticCVClassifier pkg=MLJScikitLearnInterface verbosity=0
RandomForestClassifier = @load RandomForestClassifier pkg=MLJScikitLearnInterface verbosity=0
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux verbosity=0




MLJFlux.NeuralNetworkClassifier

### Funciones auxiliares

In [17]:
function training_sample_append(cv_split, test_sample_index)
        training_indices = []
        for vector in cv_split[Not(test_sample_index)]
                training_indices = [training_indices; vector]
        end
        return training_indices, cv_split[test_sample_index]
end

training_sample_append (generic function with 1 method)

In [18]:
function dml(x, d, y, modely, modeld, nfold; classifier=true)
        n = length(y)
        cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
        
        # Coercionar variables objetivo para MLJ
        y_mlj = y
        d_mlj = classifier ? categorical(d) : d # d es categórico para clasificadores

        machine_y = machine(modely, x, y_mlj, scitype_check_level=0)
        machine_d = machine(modeld, x, d_mlj, scitype_check_level=0)
        
        y_hat = zeros(n)
        d_hat = zeros(n)

        for fold in 1:nfold
                training_fold, test_fold = training_sample_append(cv, fold)
                
                # Entrenar y predecir Y (Resultado)
                MLJ.fit!(machine_y, rows = training_fold)
                y_hat[test_fold] = MLJ.predict(machine_y, x[test_fold, :])

                # Entrenar y predecir D (Tratamiento)
                MLJ.fit!(machine_d, rows = training_fold)
                if classifier
                    d_hat_probs = MLJ.predict(machine_d, x[test_fold, :])
                    # Obtener la probabilidad de la clase '1.0' (equivalente a predict_proba[:, 1])
                    d_hat[test_fold] = pdf.(d_hat_probs, 1.0)
                else
                    d_hat[test_fold] = MLJ.predict(machine_d, x[test_fold, :])
                end
        end

        # Regresión final: resy ~ resd
        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))
        
        ols_data = DataFrame(resy = resy, resd = resd[:, 1])
        estimate_model = lm(@formula(resy ~ resd), ols_data)
        
        # Extraer coeficiente y error estándar para 'resd' (el segundo coeficiente)
        coef_est = GLM.coef(estimate_model)[2]
        se = GLM.stderror(estimate_model)[2]
        
        println(" coef (se) = ", coef_est , " (", se, ")")
        return coef_est, se, resy, resd
end

dml (generic function with 1 method)

In [19]:
## DML NAIVE
function dml_naive(x, d, y, modely, modeld; classifier=true)
        n = length(y)
        
        y_mlj = y
        d_mlj = classifier ? categorical(d) : d

        # 1. Entrenar modely en TODOS los datos y predecir EN MUESTRA
        machine_y = machine(modely, x, y_mlj, scitype_check_level=0)
        MLJ.fit!(machine_y, rows = 1:n)
        y_hat = MLJ.predict(machine_y, x)

        # 2. Entrenar modeld en TODOS los datos y predecir EN MUESTRA
        machine_d = machine(modeld, x, d_mlj, scitype_check_level=0)
        MLJ.fit!(machine_d, rows = 1:n)
        
        d_hat = zeros(n)
        if classifier
            d_hat_probs = MLJ.predict(machine_d, x)
            d_hat = pdf.(d_hat_probs, 1.0)
        else
            d_hat = MLJ.predict(machine_d, x)
        end
        
        # 3. Calcular residuales (en muestra)
        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))

        # 4. Regresión final
        ols_data = DataFrame(resy = resy, resd = resd[:, 1])
        estimate_model = lm(@formula(resy ~ resd), ols_data)
        
        coef_est = GLM.coef(estimate_model)[2]
        se = GLM.stderror(estimate_model)[2]
        
        println(" coef (se) = ", coef_est , " (", se, ")")
        return coef_est, se, resy, resd
end

dml_naive (generic function with 1 method)

In [20]:
function summarize(point, stderr, resy, resd, name)
        return DataFrame(
                model = [name],
                estimate = [point], stderr = [stderr], 
                rmse_y = [sqrt(mean(resy .^ 2))],
                # Asegurarse que resd sea un vector para el cálculo de la media
                rmse_d = [sqrt(mean(vec(resd) .^ 2))]
        )
end

summarize (generic function with 1 method)

# Debiased ML

## DEFINIR FUNC

In [21]:
# 1. OLS / Logit
modely_ols = Standardizer() |> LinearRegressor()
modeld_ols = Standardizer() |> LogisticCVClassifier(
    cv=5, 
    random_state=123, 
    solver="liblinear" 
)

ProbabilisticPipeline(
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  logistic_cv_classifier = LogisticCVClassifier(
        Cs = 10, 
        fit_intercept = true, 
        cv = 5, 
        dual = false, 
        penalty = "l2", 
        scoring = nothing, 
        solver = "liblinear", 
        tol = 0.0001, 
        max_iter = 100, 
        class_weight = nothing, 
        n_jobs = nothing, 
        verbose = 0, 
        refit = true, 
        intercept_scaling = 1.0, 
        multi_class = "auto", 
        random_state = 123, 
        l1_ratios = nothing), 
  cache = true)

In [22]:
# 2. Lasso
modely_lasso = Standardizer() |> LassoCVRegressor(cv=5, random_state=123)
modeld_lasso = Standardizer() |> LogisticCVClassifier(
    cv=5, 
    penalty="l1",      
    solver="liblinear",  
    random_state=123
)

ProbabilisticPipeline(
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  logistic_cv_classifier = LogisticCVClassifier(
        Cs = 10, 
        fit_intercept = true, 
        cv = 5, 
        dual = false, 
        penalty = "l1", 
        scoring = nothing, 
        solver = "liblinear", 
        tol = 0.0001, 
        max_iter = 100, 
        class_weight = nothing, 
        n_jobs = nothing, 
        verbose = 0, 
        refit = true, 
        intercept_scaling = 1.0, 
        multi_class = "auto", 
        random_state = 123, 
        l1_ratios = nothing), 
  cache = true)

In [23]:
# 3. Random Forest
modely_rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123)
modeld_rf = RandomForestClassifier(n_estimators=100, min_samples_leaf=5, random_state=123)

RandomForestClassifier(
  n_estimators = 100, 
  criterion = "gini", 
  max_depth = nothing, 
  min_samples_split = 2, 
  min_samples_leaf = 5, 
  min_weight_fraction_leaf = 0.0, 
  max_features = "sqrt", 
  max_leaf_nodes = nothing, 
  min_impurity_decrease = 0.0, 
  bootstrap = true, 
  oob_score = false, 
  n_jobs = nothing, 
  random_state = 123, 
  verbose = 0, 
  warm_start = false, 
  class_weight = nothing, 
  ccp_alpha = 0.0, 
  max_samples = nothing, 
  monotonic_cst = nothing)

In [24]:
# 4. Neural Network (NN)
builder_reg = MLJFlux.MLP(hidden=(50, 20), σ=relu)
modely_nn = Standardizer() |> NeuralNetworkRegressor(
    builder = builder_reg, 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 123
)

DeterministicPipeline(
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  neural_network_regressor = NeuralNetworkRegressor(
        builder = MLP(hidden = (50, 20), …), 
        optimiser = Adam(eta=0.001, beta=(0.9, 0.999), epsilon=1.0e-8), 
        loss = Flux.Losses.mse, 
        epochs = 100, 
        batch_size = 32, 
        lambda = 0.0, 
        alpha = 0.0, 
        rng = 123, 
        optimiser_changes_trigger_retraining = false, 
        acceleration = CPU1{Nothing}(nothing), 
        embedding_dims = Dict{Symbol, Real}()), 
  cache = true)

In [25]:
builder_clf = MLJFlux.MLP(hidden=(50, 20), σ=relu)
modeld_nn = Standardizer() |> NeuralNetworkClassifier(
    builder = builder_clf, 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 123
)

ProbabilisticPipeline(
  standardizer = Standardizer(
        features = Symbol[], 
        ignore = false, 
        ordered_factor = false, 
        count = false), 
  neural_network_classifier = NeuralNetworkClassifier(
        builder = MLP(hidden = (50, 20), …), 
        finaliser = NNlib.softmax, 
        optimiser = Adam(eta=0.001, beta=(0.9, 0.999), epsilon=1.0e-8), 
        loss = Flux.Losses.crossentropy, 
        epochs = 100, 
        batch_size = 32, 
        lambda = 0.0, 
        alpha = 0.0, 
        rng = 123, 
        optimiser_changes_trigger_retraining = false, 
        acceleration = CPU1{Nothing}(nothing), 
        embedding_dims = Dict{Symbol, Real}()), 
  cache = true)

## EJECUTAR DML

In [26]:
println("\n OLS/Logit ")
result_OLS = dml(x, d, y, modely_ols, modeld_ols, 10, classifier=true)
table_OLS = summarize(result_OLS..., "OLS/Logit")

println("\n Lasso ")
result_Lasso = dml(x, d, y, modely_lasso, modeld_lasso, 10, classifier=true)
table_Lasso = summarize(result_Lasso..., "Lasso")

println("\n Random Forest ")
result_RF = dml(x, d, y, modely_rf, modeld_rf, 10, classifier=true)
table_RF = summarize(result_RF..., "Random Forest")

println("\n NN (MLP) ")
result_NN = dml(x, d, y, modely_nn, modeld_nn, 10, classifier=true)
table_NN = summarize(result_NN..., "NN (MLP)")

#  4.3 Combinar y Mostrar Tabla  
table_dml = vcat(table_OLS, table_Lasso, table_RF, table_NN)
table_dml_sorted = sort(table_dml, [:rmse_y, :rmse_d])

pretty_table(table_dml_sorted)


 OLS/Logit 


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:linear_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:logistic_cv_classifier, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
C:\Users\user2\.julia\environments\v1.12\.CondaPkg\.pixi\envs\default\Lib\site-packages\sklearn\linear_model\_logistic.py:1905: Futu

 coef (se) = -0.06828218690904571 (0.03494935235373572)

 Lasso 


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:lasso_cv_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:logistic_cv_classifier, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
C:\Users\user2\.julia\environments\v1.12\.CondaPkg\.pixi\envs\default\Lib\site-packages\sklearn\linear_model\_logistic.py:1905: Fu

 coef (se) = -0.06791344479902942 (0.03495011948265716)

 Random Forest 


┌ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestClassifier(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestClassifier(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestClassifier(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
└ @ MLJBas

 coef (se) = -0.07960873374081812 (0.035415910848292494)

 NN (MLP) 


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:neural_network_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: MLJFlux: converting input data to Float32
└ @ MLJFlux C:\Users\user2\.julia\packages\MLJFlux\5eWpt\src\core.jl:294
Optimising neural net: 100%[=========================] Time: 0:00:27
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:neural_network_classifier, …).
└ @ ML

 coef (se) = -0.08871607740301986 (0.034779062506370396)
┌───────────────┬────────────┬───────────┬─────────┬──────────┐
│         model │   estimate │    stderr │  rmse_y │   rmse_d │
│        String │    Float64 │   Float64 │ Float64 │  Float64 │
├───────────────┼────────────┼───────────┼─────────┼──────────┤
│         Lasso │ -0.0679134 │ 0.0349501 │ 1.19588 │ 0.495465 │
│     OLS/Logit │ -0.0682822 │ 0.0349494 │ 1.19593 │ 0.487134 │
│ Random Forest │ -0.0796087 │ 0.0354159 │ 1.21201 │ 0.479109 │
│      NN (MLP) │ -0.0887161 │ 0.0347791 │ 1.24793 │ 0.502266 │
└───────────────┴────────────┴───────────┴─────────┴──────────┘


# Debiased ML (sin cross fitting)

In [27]:
println("\nOLS/Logit (Naive) ---")
result_OLS_naive = dml_naive(x, d, y, modely_ols, modeld_ols, classifier=true)
table_OLS_naive = summarize(result_OLS_naive..., "OLS/Logit")

println("\nLasso (Naive) ---")
result_Lasso_naive = dml_naive(x, d, y, modely_lasso, modeld_lasso, classifier=true)
table_Lasso_naive = summarize(result_Lasso_naive..., "Lasso")

println("\nRandom Forest (Naive) ---")
result_RF_naive = dml_naive(x, d, y, modely_rf, modeld_rf, classifier=true)
table_RF_naive = summarize(result_RF_naive..., "Random Forest")

println("\nNN (MLP) (Naive) ---")
result_NN_naive = dml_naive(x, d, y, modely_nn, modeld_nn, classifier=true)
table_NN_naive = summarize(result_NN_naive..., "NN (MLP)")

# Combinar y Mostrar Tabla 
table_naive = vcat(table_OLS_naive, table_Lasso_naive, table_RF_naive, table_NN_naive)
table_naive_sorted = sort(table_naive, [:rmse_y, :rmse_d])

pretty_table(table_naive_sorted)


OLS/Logit (Naive) ---


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:linear_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:logistic_cv_classifier, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
C:\Users\user2\.julia\environments\v1.12\.CondaPkg\.pixi\envs\default\Lib\site-packages\sklearn\linear_model\_logistic.py:1905: Futu

 coef (se) = -0.07231208325663993 (0.035147618160793474)

Lasso (Naive) ---


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:lasso_cv_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:logistic_cv_classifier, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
C:\Users\user2\.julia\environments\v1.12\.CondaPkg\.pixi\envs\default\Lib\site-packages\sklearn\linear_model\_logistic.py:1905: Fu

 coef (se) = -0.07319468093596589 (0.03513484275126338)

Random Forest (Naive) ---


┌ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(RandomForestClassifier(n_estimators = 100, …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499


 coef (se) = -0.07598008605885942 (0.0345611884015284)

NN (MLP) (Naive) ---


┌ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:neural_network_regressor, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: MLJFlux: converting input data to Float32
└ @ MLJFlux C:\Users\user2\.julia\packages\MLJFlux\5eWpt\src\core.jl:294
Optimising neural net: 100%[=========================] Time: 0:00:03
┌ Info: Training machine(ProbabilisticPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:standardizer, …).
└ @ MLJBase C:\Users\user2\.julia\packages\MLJBase\GY2fM\src\machines.jl:499
┌ Info: Training machine(:neural_network_classifier, …).
└ @ ML

 coef (se) = -0.08518180596029706 (0.03530504421060281)
┌───────────────┬────────────┬───────────┬─────────┬──────────┐
│         model │   estimate │    stderr │  rmse_y │   rmse_d │
│        String │    Float64 │   Float64 │ Float64 │  Float64 │
├───────────────┼────────────┼───────────┼─────────┼──────────┤
│      NN (MLP) │ -0.0851818 │  0.035305 │ 1.11211 │ 0.441251 │
│ Random Forest │ -0.0759801 │ 0.0345612 │ 1.12841 │ 0.457104 │
│     OLS/Logit │ -0.0723121 │ 0.0351476 │ 1.19047 │ 0.494446 │
│         Lasso │ -0.0731947 │ 0.0351348 │ 1.19062 │      0.5 │
└───────────────┴────────────┴───────────┴─────────┴──────────┘


# comparar tablas

In [28]:
table_dml_copy = copy(table_dml)
table_naive_copy = copy(table_naive)

table_dml_copy[!, :Method] .= "DML (Cross-Fit)"
table_naive_copy[!, :Method] .= "Naive (No Cross-Fit)"

comparison = vcat(table_dml_copy, table_naive_copy)

comparison_sorted = sort(comparison, :model)
CSV.write(output_path, comparison_sorted)
pretty_table(comparison_sorted)

┌───────────────┬────────────┬───────────┬─────────┬──────────┬──────────────────────┐
│         model │   estimate │    stderr │  rmse_y │   rmse_d │               Method │
│        String │    Float64 │   Float64 │ Float64 │  Float64 │               String │
├───────────────┼────────────┼───────────┼─────────┼──────────┼──────────────────────┤
│         Lasso │ -0.0679134 │ 0.0349501 │ 1.19588 │ 0.495465 │      DML (Cross-Fit) │
│         Lasso │ -0.0731947 │ 0.0351348 │ 1.19062 │      0.5 │ Naive (No Cross-Fit) │
│      NN (MLP) │ -0.0887161 │ 0.0347791 │ 1.24793 │ 0.502266 │      DML (Cross-Fit) │
│      NN (MLP) │ -0.0851818 │  0.035305 │ 1.11211 │ 0.441251 │ Naive (No Cross-Fit) │
│     OLS/Logit │ -0.0682822 │ 0.0349494 │ 1.19593 │ 0.487134 │      DML (Cross-Fit) │
│     OLS/Logit │ -0.0723121 │ 0.0351476 │ 1.19047 │ 0.494446 │ Naive (No Cross-Fit) │
│ Random Forest │ -0.0796087 │ 0.0354159 │ 1.21201 │ 0.479109 │      DML (Cross-Fit) │
│ Random Forest │ -0.0759801 │ 0.0345612 │ 

### Pregunta 1: What can you say about the RMSE for predicting y and d?

**Respuesta:** Al ver la tabla final, se ve claramente que los valores de `rmse_y` y `rmse_d` son  más bajos en el método "Naive" que en el método "DML (Cross-Fit)". Esta diferencia es mucho más pronunciada en los modelos flexibles como Random Forest y NN. Esto es por el overfitting!!

### Pregunta 2: Why is it that estimating with one function yields lower RMSE than another?

**Respuesta:** Esto se debe al **sobreajuste (overfitting)**.

* La función **Naive (`dml_naive`)** entrena y evalúa el modelo en el  misma data. Los modelos complejos (especialmente RF y NN) son muy buenos para "memorizar" los datos de entrenamiento, incluido el noise. Esto da como resultado un RMSE artificialmente bajo y excesivamente optimista
* La función **DML (`dml`)** usa **cross-fitting (ajuste cruzado)**. Entrena el modelo en una parte de los datos(pliegues) y lo evalúa en una parte que no se ha visto anteriormente. Este RMSE de cross fitting es una medida válida y realista del poder predictivo del modelo en datos nuevos.

### Pregunta 3: What problem would we have if we chose to estimate without cross-fitting?

**Respuesta:** El problema principal es el **sesgo por sobreajuste (overfitting**.

La teoría de DML (Double Machine Learning) requiere que los residuos (`resy` y `resD`) se generen de una manera que sea "ortogonal" (estadísticamente independiente) del proceso de estimación. El **cross-fitting** es el mecanismo que nos permite lograr esto.

En resumen, **sin cross-fitting, sacrificamos la validez y la inferencia causal correcta por un falso sentido de precisión (de un RMSE más bajo) que proviene del overfitting, pero que de probarlo con data no "aprendida" resultara en estimaciones erroneas por overfitting**